In [1]:
import tensorflow as tf
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Load data
file_path = Path("Resources/final_sample.csv")
df = pd.read_csv(file_path)
df.head()

,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,applicant_ethnicity,co_applicant_ethnicity,...,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,Region,action_taken_summary
0,NCUA,Conventional,One to Four-Family,Refinancing,Not Owner Occupied,123.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,2310.0,20.35,65300.0,74.28,941.0,3848.0,MW,0
1,HUD,Conventional,One to Four-Family,Home Purchase,Owner Occupied,387.0,Not Applicable,Application Approved but not Accepted,Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,5192.0,96.51,64300.0,53.53,397.0,1056.0,W,0
2,HUD,Conventional,One to Four-Family,Refinancing,Owner Occupied,376.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,12130.0,66.65,63200.0,112.07,921.0,2099.0,W,0
3,HUD,FHA-Insured,One to Four-Family,Refinancing,Owner Occupied,272.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,6335.0,27.73,91500.0,136.37,1406.0,1541.0,MA,0
4,NCUA,Conventional,One to Four-Family,Home Improvement,Owner Occupied,40.0,Not Applicable,Application Approved but not Accepted,Info not Provided by Applicant,No Co-Applicant,...,Not a HOEPA Loan,Secured by a Subordinate Loan,5625.0,5.33,79600.0,153.58,1693.0,1679.0,MW,0


In [3]:
df = df.drop(columns=["purchaser_type"])
df.head(10)

,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,applicant_ethnicity,co_applicant_ethnicity,...,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,Region,action_taken_summary
0,NCUA,Conventional,One to Four-Family,Refinancing,Not Owner Occupied,123.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,2310.0,20.35,65300.0,74.28,941.0,3848.0,MW,0
1,HUD,Conventional,One to Four-Family,Home Purchase,Owner Occupied,387.0,Not Applicable,Application Approved but not Accepted,Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,5192.0,96.51,64300.0,53.53,397.0,1056.0,W,0
2,HUD,Conventional,One to Four-Family,Refinancing,Owner Occupied,376.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,12130.0,66.65,63200.0,112.07,921.0,2099.0,W,0
3,HUD,FHA-Insured,One to Four-Family,Refinancing,Owner Occupied,272.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,Not Hispanic or Latino,...,Not a HOEPA Loan,Secured by First Lien,6335.0,27.73,91500.0,136.37,1406.0,1541.0,MA,0
4,NCUA,Conventional,One to Four-Family,Home Improvement,Owner Occupied,40.0,Not Applicable,Application Approved but not Accepted,Info not Provided by Applicant,No Co-Applicant,...,Not a HOEPA Loan,Secured by a Subordinate Loan,5625.0,5.33,79600.0,153.58,1693.0,1679.0,MW,0
5,FDIC,Conventional,One to Four-Family,Home Purchase,Owner Occupied,104.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,4139.0,51.65,64000.0,42.41,426.0,1328.0,NW,0
6,HUD,FHA-Insured,One to Four-Family,Refinancing,Owner Occupied,315.0,Not Applicable,Application Approved but not Accepted,Info not Provided by Applicant,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,4095.0,53.28,67900.0,221.47,972.0,1129.0,SE,0
7,HUD,Conventional,One to Four-Family,Refinancing,Owner Occupied,492.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,6180.0,10.61,91100.0,125.65,2084.0,2233.0,MA,0
8,OCC,Conventional,One to Four-Family,Home Purchase,Owner Occupied,105.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,4503.0,21.81,61500.0,125.60,1559.0,1805.0,MW,0
9,NCUA,Conventional,One to Four-Family,Home Purchase,Owner Occupied,400.0,Not Applicable,Application Approved but not Accepted,Not Hispanic or Latino,No Co-Applicant,...,Not a HOEPA Loan,Secured by First Lien,5710.0,13.77,68000.0,131.23,1556.0,2104.0,MA,0


In [4]:
# check counts
df.count()

agency_code                       100725
loan_type                         100725
property_type                     100725
loan_purpose                      100725
owner_occupancy                   100725
loan_amount_000s                  100725
preapproval                       100725
action_taken                      100725
applicant_ethnicity               100725
co_applicant_ethnicity            100725
applicant_race_1                  100725
co_applicant_race_1               100725
applicant_sex                     100725
co_applicant_sex                  100725
applicant_income_000s             100725
hoepa_status                      100725
lien_status                       100725
population                        100725
minority_population               100725
hud_median_family_income          100725
tract_to_msamd_income             100725
number_of_owner_occupied_units    100725
number_of_1_to_4_family_units     100725
Region                            100725
action_taken_sum

In [5]:
df.agency_code.value_counts()

HUD     47189
CFPB    30285
FDIC     8757
NCUA     7893
OCC      3636
FRS      2965
Name: agency_code, dtype: int64

In [6]:
# drop action taken column
df = df.drop(columns=["action_taken"])

In [7]:
# check df again
df.head(10)

,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,applicant_ethnicity,co_applicant_ethnicity,applicant_race_1,...,hoepa_status,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,Region,action_taken_summary
0,NCUA,Conventional,One to Four-Family,Refinancing,Not Owner Occupied,123.0,Not Applicable,Not Hispanic or Latino,Not Hispanic or Latino,White,...,Not a HOEPA Loan,Secured by First Lien,2310.0,20.35,65300.0,74.28,941.0,3848.0,MW,0
1,HUD,Conventional,One to Four-Family,Home Purchase,Owner Occupied,387.0,Not Applicable,Hispanic or Latino,Not Hispanic or Latino,White,...,Not a HOEPA Loan,Secured by First Lien,5192.0,96.51,64300.0,53.53,397.0,1056.0,W,0
2,HUD,Conventional,One to Four-Family,Refinancing,Owner Occupied,376.0,Not Applicable,Not Hispanic or Latino,Not Hispanic or Latino,Asian,...,Not a HOEPA Loan,Secured by First Lien,12130.0,66.65,63200.0,112.07,921.0,2099.0,W,0
3,HUD,FHA-Insured,One to Four-Family,Refinancing,Owner Occupied,272.0,Not Applicable,Not Hispanic or Latino,Not Hispanic or Latino,White,...,Not a HOEPA Loan,Secured by First Lien,6335.0,27.73,91500.0,136.37,1406.0,1541.0,MA,0
4,NCUA,Conventional,One to Four-Family,Home Improvement,Owner Occupied,40.0,Not Applicable,Info not Provided by Applicant,No Co-Applicant,Information not Provided,...,Not a HOEPA Loan,Secured by a Subordinate Loan,5625.0,5.33,79600.0,153.58,1693.0,1679.0,MW,0
5,FDIC,Conventional,One to Four-Family,Home Purchase,Owner Occupied,104.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,4139.0,51.65,64000.0,42.41,426.0,1328.0,NW,0
6,HUD,FHA-Insured,One to Four-Family,Refinancing,Owner Occupied,315.0,Not Applicable,Info not Provided by Applicant,No Co-Applicant,Information not Provided,...,Not a HOEPA Loan,Secured by First Lien,4095.0,53.28,67900.0,221.47,972.0,1129.0,SE,0
7,HUD,Conventional,One to Four-Family,Refinancing,Owner Occupied,492.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,6180.0,10.61,91100.0,125.65,2084.0,2233.0,MA,0
8,OCC,Conventional,One to Four-Family,Home Purchase,Owner Occupied,105.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,4503.0,21.81,61500.0,125.60,1559.0,1805.0,MW,0
9,NCUA,Conventional,One to Four-Family,Home Purchase,Owner Occupied,400.0,Not Applicable,Not Hispanic or Latino,No Co-Applicant,White,...,Not a HOEPA Loan,Secured by First Lien,5710.0,13.77,68000.0,131.23,1556.0,2104.0,MA,0


Run Neural Network

In [8]:
# Generate our categorical variable lists
action_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df[action_cat].nunique()

agency_code               6
loan_type                 4
property_type             2
loan_purpose              3
owner_occupancy           3
preapproval               3
applicant_ethnicity       4
co_applicant_ethnicity    5
applicant_race_1          7
co_applicant_race_1       8
applicant_sex             4
co_applicant_sex          5
hoepa_status              2
lien_status               4
Region                    7
dtype: int64

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[action_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(action_cat)
encode_df.head()

,agency_code_CFPB,agency_code_FDIC,agency_code_FRS,agency_code_HUD,agency_code_NCUA,agency_code_OCC,loan_type_Conventional,loan_type_FHA-Insured,loan_type_FSA/RHS,loan_type_VA-Guaranteed,...,lien_status_Not Secured by Lien,lien_status_Secured by First Lien,lien_status_Secured by a Subordinate Loan,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
df = df.merge(encode_df,left_index=True, right_index=True)
df = df.drop(action_cat,1)
df.head()

c:\Users\olivi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,loan_amount_000s,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,action_taken_summary,agency_code_CFPB,...,lien_status_Not Secured by Lien,lien_status_Secured by First Lien,lien_status_Secured by a Subordinate Loan,Region_MA,Region_MW,Region_NE,Region_NW,Region_SE,Region_SW,Region_W
0,123.0,187.0,2310.0,20.35,65300.0,74.28,941.0,3848.0,0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,387.0,148.0,5192.0,96.51,64300.0,53.53,397.0,1056.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,376.0,208.0,12130.0,66.65,63200.0,112.07,921.0,2099.0,0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,272.0,111.0,6335.0,27.73,91500.0,136.37,1406.0,1541.0,0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40.0,120.0,5625.0,5.33,79600.0,153.58,1693.0,1679.0,0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# check values
df.count()

loan_amount_000s            100725
applicant_income_000s       100725
population                  100725
minority_population         100725
hud_median_family_income    100725
                             ...  
Region_NE                   100725
Region_NW                   100725
Region_SE                   100725
Region_SW                   100725
Region_W                    100725
Length: 76, dtype: int64

In [12]:
# Split our preprocessed data into our features and target arrays
y = df["action_taken_summary"].values
X = df.drop(["action_taken_summary"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

c:\Users\olivi\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
import keras

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 76
hidden_nodes_layer2 = 51
hidden_nodes_layer3 = 30
hidden_nodes_layer4 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

## Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

## Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 76)                5776      
                                                                 
 dense_1 (Dense)             (None, 51)                3927      
                                                                 
 dense_2 (Dense)             (None, 30)                1560      
                                                                 
 dense_3 (Dense)             (None, 20)                620       
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 11,904
Trainable params: 11,904
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="Adadelta", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
2361/2361 [==============================] - 5s 2ms/step - loss: 266.4642 - accuracy: 0.5315
Epoch 2/50
2361/2361 [==============================] - 4s 2ms/step - loss: 4.8986 - accuracy: 0.6720
Epoch 3/50
2361/2361 [==============================] - 4s 2ms/step - loss: 3.4358 - accuracy: 0.6799
Epoch 4/50
2361/2361 [==============================] - 4s 2ms/step - loss: 2.3232 - accuracy: 0.6877
Epoch 5/50
2361/2361 [==============================] - 4s 2ms/step - loss: 1.8503 - accuracy: 0.6924
Epoch 6/50
2361/2361 [==============================] - 4s 2ms/step - loss: 1.5823 - accuracy: 0.6937
Epoch 7/50
2361/2361 [==============================] - 4s 2ms/step - loss: 1.3945 - accuracy: 0.6988
Epoch 8/50
2361/2361 [==============================] - 4s 2ms/step - loss: 1.2630 - accuracy: 0.7007
Epoch 9/50
2361/2361 [==============================] - 4s 2ms/step - loss: 1.1705 - accuracy: 0.7046
Epoch 10/50
2361/2361 [==============================] - 4s 2ms/step - loss: 1.0

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

787/787 - 1s - loss: 0.7392 - accuracy: 0.3684 - 1s/epoch - 1ms/step
Loss: 0.7392458915710449, Accuracy: 0.3683583438396454
